# **GRU**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# =========================================================
# 0) 평가 지표
# =========================================================
def smape(a, f):
    a = np.array(a)
    f = np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    mask = denom != 0
    return np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100

def evaluate(actual, pred):
    actual = np.array(actual)
    pred = np.array(pred)

    mae = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred)**2))
    smape_v = smape(actual, pred)

    return mae, rmse, smape_v


# =========================================================
# 1) 매출 정제
# =========================================================
def clean_sales(df):
    df['일매출'] = (
        df['일매출'].astype(str)
        .str.replace(',', '')
        .str.replace(' ', '')
        .str.strip()
    )
    df['일매출'] = pd.to_numeric(df['일매출'], errors='coerce').fillna(0)
    return df

# =========================================================
# 2) 날짜 기반 피처 생성
# =========================================================
def add_date_features(df):
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['weekday'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)

    # 운영시간
    df['open_hours'] = 11  # 월~금
    df.loc[df['weekday'] == 5, 'open_hours'] = 6  # 토
    df.loc[df['weekday'] == 6, 'open_hours'] = 0  # 일

    return df

# =========================================================
# 3) Lag / Rolling Feature 생성
# =========================================================
def add_lag_features(df):
    df['lag1'] = df['일매출'].shift(1)
    df['lag2'] = df['일매출'].shift(2)
    df['lag3'] = df['일매출'].shift(3)
    df['lag7'] = df['일매출'].shift(7)
    df['lag14'] = df['일매출'].shift(14)
    df['lag28'] = df['일매출'].shift(28)

    df['roll_mean7'] = df['일매출'].rolling(7).mean()
    df['roll_mean14'] = df['일매출'].rolling(14).mean()
    df['roll_mean28'] = df['일매출'].rolling(28).mean()

    df['roll_std7'] = df['일매출'].rolling(7).std()
    df['roll_std28'] = df['일매출'].rolling(28).std()

    return df

# =========================================================
# 4) Dataset
# =========================================================
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# =========================================================
# 5) GRU 모델
# =========================================================
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

# =========================================================
# 6) Autoregressive 예측
# =========================================================
def autoregressive_forecast(model, df_train, df_test, feature_cols, scaler_X, scaler_y):
    history = df_train.copy()
    predictions = []

    for i in range(len(df_test)):
        row = df_test.iloc[i].copy()
        last = history['일매출'].values

        # ---- lag/rolling 업데이트 ----
        row['lag1'] = last[-1]
        row['lag2'] = last[-2]
        row['lag3'] = last[-3]
        row['lag7'] = last[-7]
        row['lag14'] = last[-14]
        row['lag28'] = last[-28]

        row['roll_mean7'] = pd.Series(last[-7:]).mean()
        row['roll_mean14'] = pd.Series(last[-14:]).mean()
        row['roll_mean28'] = pd.Series(last[-28:]).mean()

        row['roll_std7'] = pd.Series(last[-7:]).std()
        row['roll_std28'] = pd.Series(last[-28:]).std()

        # ---- scaling ----
        X_input = scaler_X.transform([row[feature_cols].values])
        X_tensor = torch.tensor(X_input, dtype=torch.float32).unsqueeze(1)

        pred_scaled = model(X_tensor).item()
        pred = scaler_y.inverse_transform([[pred_scaled]])[0][0]

        pred = max(pred, 0)  # 음수 방지
        predictions.append(pred)

        # history 업데이트
        new_row = row.copy()
        new_row['일매출'] = pred
        history = pd.concat([history, pd.DataFrame([new_row])], ignore_index=True)

    return predictions


# =========================================================
# =============== 🔥 메인 실행 ===========================
# =========================================================

train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test['영업일자'] = pd.to_datetime(test['영업일자'])

# --- 전처리 ---
train = clean_sales(train)
test = clean_sales(test)

train = add_date_features(train)
test = add_date_features(test)

train = add_lag_features(train)
test = add_lag_features(test)

train = train.dropna().reset_index(drop=True)

# --- Scaling ---
target_col = '일매출'
feature_cols = [
    'month','day','weekday','is_weekend','open_hours',
    'lag1','lag2','lag3','lag7','lag14','lag28',
    'roll_mean7','roll_mean14','roll_mean28',
    'roll_std7','roll_std28'
]

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

train[feature_cols] = scaler_X.fit_transform(train[feature_cols])
train[[target_col]] = scaler_y.fit_transform(train[[target_col]])

# --- GRU 학습 ---
X_train = train[feature_cols].values.reshape(-1, 1, len(feature_cols))
y_train = train[target_col].values

dataset = SequenceDataset(X_train, y_train)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

model = GRUModel(input_dim=len(feature_cols))

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(40):
    total_loss = 0
    for Xb, yb in loader:
        optimizer.zero_grad()
        pred = model(Xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[Epoch {epoch+1}/40] Loss: {total_loss:.4f}")

# --- Test 예측 ---
test_pred = autoregressive_forecast(model, train, test, feature_cols, scaler_X, scaler_y)

# --- 평가지표 계산 ---
actual_test = test['일매출'].values
mae, rmse, smape_val = evaluate(actual_test, test_pred)

print("\n------ Test 성능 (GRU + Feature) ------")
print(f"MAE   : {mae:.3f}")
print(f"RMSE  : {rmse:.3f}")
print(f"SMAPE : {smape_val:.3f}%")

test['예측매출'] = test_pred
print("\n------ Test 예측 결과 ------")
print(test[['영업일자', '예측매출']])


[Epoch 1/40] Loss: 0.7621
[Epoch 2/40] Loss: 0.3673
[Epoch 3/40] Loss: 0.3121
[Epoch 4/40] Loss: 0.2962
[Epoch 5/40] Loss: 0.2858
[Epoch 6/40] Loss: 0.2711
[Epoch 7/40] Loss: 0.2780
[Epoch 8/40] Loss: 0.2671
[Epoch 9/40] Loss: 0.2559
[Epoch 10/40] Loss: 0.2488
[Epoch 11/40] Loss: 0.2561
[Epoch 12/40] Loss: 0.2419
[Epoch 13/40] Loss: 0.2482
[Epoch 14/40] Loss: 0.2494
[Epoch 15/40] Loss: 0.2363
[Epoch 16/40] Loss: 0.2289
[Epoch 17/40] Loss: 0.2340
[Epoch 18/40] Loss: 0.2279
[Epoch 19/40] Loss: 0.2326
[Epoch 20/40] Loss: 0.2324
[Epoch 21/40] Loss: 0.2266
[Epoch 22/40] Loss: 0.2155
[Epoch 23/40] Loss: 0.2169
[Epoch 24/40] Loss: 0.2183
[Epoch 25/40] Loss: 0.2187
[Epoch 26/40] Loss: 0.2130
[Epoch 27/40] Loss: 0.2034
[Epoch 28/40] Loss: 0.2064
[Epoch 29/40] Loss: 0.2064
[Epoch 30/40] Loss: 0.2170
[Epoch 31/40] Loss: 0.2043
[Epoch 32/40] Loss: 0.2073
[Epoch 33/40] Loss: 0.2045
[Epoch 34/40] Loss: 0.1927
[Epoch 35/40] Loss: 0.1992
[Epoch 36/40] Loss: 0.2088
[Epoch 37/40] Loss: 0.2267
[Epoch 38/

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valida


------ Test 성능 (GRU + Feature) ------
MAE   : 68079.590
RMSE  : 102478.171
SMAPE : 83.310%

------ Test 예측 결과 ------
         영업일자           예측매출
0  2025-08-10       0.000000
1  2025-08-11   20510.521701
2  2025-08-12   27092.345254
3  2025-08-13   31875.100745
4  2025-08-14   36480.008759
..        ...            ...
78 2025-10-27  303802.717718
79 2025-10-28  321190.291160
80 2025-10-29  302914.395761
81 2025-10-30  273772.321200
82 2025-10-31  255170.774782

[83 rows x 2 columns]


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valida

**GRU+학사일정**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random

# =========================================================
# 🔒 0) Seed 고정 (추가됨)
# =========================================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)


# =========================================================
# 1) 평가 지표
# =========================================================
def smape(a, f):
    a = np.array(a)
    f = np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    mask = denom != 0
    return np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100

def evaluate(actual, pred):
    actual = np.array(actual)
    pred = np.array(pred)

    mae = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred)**2))
    smape_v = smape(actual, pred)

    return mae, rmse, smape_v


# =========================================================
# 2) 매출 정제
# =========================================================
def clean_sales(df):
    df['일매출'] = (
        df['일매출'].astype(str)
        .str.replace(',', '')
        .str.replace(' ', '')
        .str.strip()
    )
    df['일매출'] = pd.to_numeric(df['일매출'], errors='coerce').fillna(0)
    return df


# =========================================================
# 3) 날짜 기반 feature
# =========================================================
def add_date_features(df):
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['weekday'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)

    df['open_hours'] = 11
    df.loc[df['weekday'] == 5, 'open_hours'] = 6
    df.loc[df['weekday'] == 6, 'open_hours'] = 0

    return df


# =========================================================
# 4) Lag / Rolling Feature
# =========================================================
def add_lag_features(df):
    df['lag1'] = df['일매출'].shift(1)
    df['lag2'] = df['일매출'].shift(2)
    df['lag3'] = df['일매출'].shift(3)
    df['lag7'] = df['일매출'].shift(7)
    df['lag14'] = df['일매출'].shift(14)
    df['lag28'] = df['일매출'].shift(28)

    df['roll_mean7'] = df['일매출'].rolling(7).mean()
    df['roll_mean14'] = df['일매출'].rolling(14).mean()
    df['roll_mean28'] = df['일매출'].rolling(28).mean()

    df['roll_std7'] = df['일매출'].rolling(7).std()
    df['roll_std28'] = df['일매출'].rolling(28).std()

    return df


# =========================================================
# 5) Dataset
# =========================================================
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# =========================================================
# 6) GRU 모델
# =========================================================
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out


# =========================================================
# 7) Autoregressive 예측
# =========================================================
def autoregressive_forecast(model, df_train, df_test, feature_cols, scaler_X, scaler_y):
    history = df_train.copy()
    predictions = []

    for i in range(len(df_test)):
        row = df_test.iloc[i].copy()
        last = history['일매출'].values

        row['lag1'] = last[-1]
        row['lag2'] = last[-2]
        row['lag3'] = last[-3]
        row['lag7'] = last[-7]
        row['lag14'] = last[-14]
        row['lag28'] = last[-28]

        row['roll_mean7'] = pd.Series(last[-7:]).mean()
        row['roll_mean14'] = pd.Series(last[-14:]).mean()
        row['roll_mean28'] = pd.Series(last[-28:]).mean()
        row['roll_std7'] = pd.Series(last[-7:]).std()
        row['roll_std28'] = pd.Series(last[-28:]).std()

        X_input = scaler_X.transform([row[feature_cols].values])
        X_tensor = torch.tensor(X_input, dtype=torch.float32).unsqueeze(1)

        pred_scaled = model(X_tensor).item()
        pred = scaler_y.inverse_transform([[pred_scaled]])[0][0]
        pred = max(pred, 0)
        predictions.append(pred)

        new_row = row.copy()
        new_row['일매출'] = pred
        history = pd.concat([history, pd.DataFrame([new_row])], ignore_index=True)

    return predictions


# =========================================================
# 🔥 메인 실행
# =========================================================

# ① POS 데이터
train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test['영업일자'] = pd.to_datetime(test['영업일자'])

train = clean_sales(train)
test = clean_sales(test)

# ② 학사일정
academic = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/학사일정_정리(2325).csv')

academic['영업일자'] = pd.to_datetime(academic['date'])
academic = academic.drop(columns=['date'])

weekday_map = {'mon':0,'tue':1,'wed':2,'thu':3,'fri':4,'sat':5,'sun':6}
academic['acad_weekday'] = academic['weekday'].map(weekday_map)
academic = academic.drop(columns=['weekday'])

academic = academic.rename(columns={
    'weekend': 'acad_weekend',
    'holiday': 'acad_holiday',
    'semester': 'acad_semester',
    'seasonal': 'acad_seasonal',
    'exam': 'acad_exam',
    'ceremony': 'acad_ceremony',
    'dormitory': 'acad_dormitory'
})

train = train.merge(academic, on='영업일자', how='left')
test = test.merge(academic, on='영업일자', how='left')

# 날짜 feature
train = add_date_features(train)
test = add_date_features(test)

# Lag/Rolling
train = add_lag_features(train)
test = add_lag_features(test)

train = train.dropna().reset_index(drop=True)

# Feature 정의
academic_cols = [
    'acad_weekday','acad_weekend','acad_holiday',
    'acad_semester','acad_seasonal','acad_exam',
    'acad_ceremony','acad_dormitory'
]

base_feature_cols = [
    'month','day','weekday','is_weekend','open_hours',
    'lag1','lag2','lag3','lag7','lag14','lag28',
    'roll_mean7','roll_mean14','roll_mean28',
    'roll_std7','roll_std28'
]

feature_cols = base_feature_cols + academic_cols

# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

train[feature_cols] = scaler_X.fit_transform(train[feature_cols])
train[['일매출']] = scaler_y.fit_transform(train[['일매출']])

# GRU 학습
X_train = train[feature_cols].values.reshape(-1, 1, len(feature_cols))
y_train = train['일매출'].values

dataset = SequenceDataset(X_train, y_train)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

model = GRUModel(input_dim=len(feature_cols))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(40):
    total_loss = 0
    for Xb, yb in loader:
        optimizer.zero_grad()
        pred = model(Xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[Epoch {epoch+1}/40] Loss: {total_loss:.4f}")

# Test 예측
test_pred = autoregressive_forecast(model, train, test, feature_cols, scaler_X, scaler_y)

# 평가
actual_test = test['일매출'].values
mae, rmse, smape_val = evaluate(actual_test, test_pred)

print("\n------ Test 성능 (GRU + Academic Feature) ------")
print(f"MAE   : {mae:.3f}")
print(f"RMSE  : {rmse:.3f}")
print(f"SMAPE : {smape_val:.3f}%")

test['예측매출'] = test_pred
print(test[['영업일자', '예측매출']])


[Epoch 1/40] Loss: 0.9982
[Epoch 2/40] Loss: 0.3847
[Epoch 3/40] Loss: 0.3155
[Epoch 4/40] Loss: 0.3171
[Epoch 5/40] Loss: 0.2777
[Epoch 6/40] Loss: 0.2571
[Epoch 7/40] Loss: 0.2417
[Epoch 8/40] Loss: 0.2313
[Epoch 9/40] Loss: 0.2308
[Epoch 10/40] Loss: 0.2329
[Epoch 11/40] Loss: 0.2343
[Epoch 12/40] Loss: 0.2188
[Epoch 13/40] Loss: 0.2208
[Epoch 14/40] Loss: 0.2109
[Epoch 15/40] Loss: 0.2211
[Epoch 16/40] Loss: 0.2063
[Epoch 17/40] Loss: 0.2182
[Epoch 18/40] Loss: 0.2045
[Epoch 19/40] Loss: 0.1946
[Epoch 20/40] Loss: 0.1985
[Epoch 21/40] Loss: 0.1983
[Epoch 22/40] Loss: 0.1874
[Epoch 23/40] Loss: 0.1874
[Epoch 24/40] Loss: 0.1872
[Epoch 25/40] Loss: 0.1997
[Epoch 26/40] Loss: 0.1943
[Epoch 27/40] Loss: 0.1789
[Epoch 28/40] Loss: 0.1804
[Epoch 29/40] Loss: 0.1986
[Epoch 30/40] Loss: 0.1774
[Epoch 31/40] Loss: 0.1802
[Epoch 32/40] Loss: 0.1843
[Epoch 33/40] Loss: 0.1756
[Epoch 34/40] Loss: 0.1734
[Epoch 35/40] Loss: 0.1720
[Epoch 36/40] Loss: 0.1704
[Epoch 37/40] Loss: 0.1700
[Epoch 38/

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valida


------ Test 성능 (GRU + Academic Feature) ------
MAE   : 99026.513
RMSE  : 133099.498
SMAPE : 79.808%
         영업일자           예측매출
0  2025-08-10       0.000000
1  2025-08-11   72975.939902
2  2025-08-12   83645.978104
3  2025-08-13   87552.034528
4  2025-08-14   88096.644589
..        ...            ...
78 2025-10-27  376321.358049
79 2025-10-28  401405.220625
80 2025-10-29  386880.434793
81 2025-10-30  347259.570721
82 2025-10-31  314919.963312

[83 rows x 2 columns]


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valida

**Feature selection**

In [ ]:
# =========================================================
# 🔍 Feature 상관관계 분석 (train 데이터 기준)
# =========================================================

# 상관관계 분석 대상 피처
corr_features = feature_cols + ['일매출']

# 상관관계 계산 (일매출과의 상관성만 추출)
corr = (
    train[corr_features]
    .corr()['일매출']           # target과의 상관성만 추출
    .drop('일매출')            # 자기 자신 제거
    .sort_values(key=lambda x: abs(x), ascending=False)   # 절대값 기준 정렬
)

print("===== 📌 Feature → 일매출 상관관계 (기여도 순 정렬) =====")
print(corr)


===== 📌 Feature → 일매출 상관관계 (기여도 순 정렬) =====
lag7              0.677263
roll_mean7        0.596336
open_hours        0.566166
is_weekend       -0.542545
acad_weekend     -0.542545
acad_semester     0.522026
roll_mean14       0.516345
lag14             0.513086
lag1              0.487982
weekday          -0.485075
acad_weekday     -0.485075
roll_std7         0.476060
lag28             0.441095
roll_mean28       0.411901
roll_std28        0.343227
acad_exam         0.249641
lag2              0.223362
acad_seasonal    -0.197854
month             0.197044
acad_dormitory    0.160147
acad_holiday     -0.153909
lag3              0.143951
acad_ceremony    -0.030401
day              -0.015447
Name: 일매출, dtype: float64


**0.5대 피처만 넣어서 실험**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random

# =========================================================
# 🔒 0) Seed 고정
# =========================================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# =========================================================
# 1) 평가 지표
# =========================================================
def smape(a, f):
    a = np.array(a)
    f = np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    mask = denom != 0
    return np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100

def evaluate(actual, pred):
    actual = np.array(actual)
    pred = np.array(pred)
    mae = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred)**2))
    smape_v = smape(actual, pred)
    return mae, rmse, smape_v


# =========================================================
# 2) 매출 정제
# =========================================================
def clean_sales(df):
    df['일매출'] = (
        df['일매출'].astype(str)
        .str.replace(',', '')
        .str.replace(' ', '')
        .str.strip()
    )
    df['일매출'] = pd.to_numeric(df['일매출'], errors='coerce').fillna(0)
    return df


# =========================================================
# 3) 날짜 기반 Feature
# =========================================================
def add_date_features(df):
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['weekday'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)

    df['open_hours'] = 11
    df.loc[df['weekday'] == 5, 'open_hours'] = 6
    df.loc[df['weekday'] == 6, 'open_hours'] = 0
    return df


# =========================================================
# 4) Lag / Rolling Feature (전체 생성 → 이후 필요한 것만 사용)
# =========================================================
def add_lag_features(df):
    df['lag7'] = df['일매출'].shift(7)
    df['lag14'] = df['일매출'].shift(14)
    df['roll_mean7'] = df['일매출'].rolling(7).mean()
    df['roll_mean14'] = df['일매출'].rolling(14).mean()
    return df


# =========================================================
# 5) Dataset
# =========================================================
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# =========================================================
# 6) GRU Model
# =========================================================
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out


# =========================================================
# 7) Autoregressive Forecast
#      → 필요한 Feature만 업데이트!
# =========================================================
def autoregressive_forecast(model, df_train, df_test, feature_cols, scaler_X, scaler_y):
    history = df_train.copy()
    predictions = []

    for i in range(len(df_test)):
        row = df_test.iloc[i].copy()
        last = history['일매출'].values

        # 필요한 Feature만 업데이트
        row['lag7'] = last[-7]
        row['lag14'] = last[-14]

        row['roll_mean7'] = pd.Series(last[-7:]).mean()
        row['roll_mean14'] = pd.Series(last[-14:]).mean()

        # 고정 Feature(open_hours, is_weekend, acad_semester)는 데이터프레임에 이미 있음

        # 스케일 변환
        X_input = scaler_X.transform([row[feature_cols].values])
        X_tensor = torch.tensor(X_input, dtype=torch.float32).unsqueeze(1)

        pred_scaled = model(X_tensor).item()
        pred = scaler_y.inverse_transform([[pred_scaled]])[0][0]

        pred = max(pred, 0)
        predictions.append(pred)

        new_row = row.copy()
        new_row['일매출'] = pred
        history = pd.concat([history, pd.DataFrame([new_row])], ignore_index=True)

    return predictions


# =========================================================
# 🔥 메인 실행
# =========================================================

# ① POS 데이터
train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test['영업일자'] = pd.to_datetime(test['영업일자'])

train = clean_sales(train)
test = clean_sales(test)

# ② 학사일정 데이터
academic = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/학사일정_정리(2325).csv')
academic['영업일자'] = pd.to_datetime(academic['date'])
academic = academic.drop(columns=['date'])

weekday_map = {'mon':0,'tue':1,'wed':2,'thu':3,'fri':4,'sat':5,'sun':6}
academic['acad_weekday'] = academic['weekday'].map(weekday_map)
academic = academic.drop(columns=['weekday'])

academic = academic.rename(columns={
    'weekend': 'acad_weekend',
    'holiday': 'acad_holiday',
    'semester': 'acad_semester',
    'seasonal': 'acad_seasonal',
    'exam': 'acad_exam',
    'ceremony': 'acad_ceremony',
    'dormitory': 'acad_dormitory'
})

train = train.merge(academic, on='영업일자', how='left')
test = test.merge(academic, on='영업일자', how='left')

# 날짜 Feature
train = add_date_features(train)
test = add_date_features(test)

# Lag / Rolling Feature (필요한 것만 만듦)
train = add_lag_features(train)
test = add_lag_features(test)

train = train.dropna().reset_index(drop=True)


# =========================================================
# 🧩 사용 Feature 7개
# =========================================================
feature_cols = [
    'lag7',
    'roll_mean7',
    'open_hours',
    'is_weekend',
    'acad_semester',
    'roll_mean14',
    'lag14'
]

# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

train[feature_cols] = scaler_X.fit_transform(train[feature_cols])
train[['일매출']] = scaler_y.fit_transform(train[['일매출']])

# GRU 학습 준비
X_train = train[feature_cols].values.reshape(-1, 1, len(feature_cols))
y_train = train['일매출'].values

dataset = SequenceDataset(X_train, y_train)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# 모델 생성
model = GRUModel(input_dim=len(feature_cols))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# GRU 학습
for epoch in range(40):
    total_loss = 0
    for Xb, yb in loader:
        optimizer.zero_grad()
        pred = model(Xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[Epoch {epoch+1}/40] Loss: {total_loss:.4f}")

# Test autoregressive 예측
test_pred = autoregressive_forecast(model, train, test, feature_cols, scaler_X, scaler_y)

# 평가
actual_test = test['일매출'].values
mae, rmse, smape_val = evaluate(actual_test, test_pred)

print("\n------ Test 성능 (GRU + Top 7 Features) ------")
print(f"MAE   : {mae:.3f}")
print(f"RMSE  : {rmse:.3f}")
print(f"SMAPE : {smape_val:.3f}%")

test['예측매출'] = test_pred
print(test[['영업일자', '예측매출']])


[Epoch 1/40] Loss: 0.7833
[Epoch 2/40] Loss: 0.4068
[Epoch 3/40] Loss: 0.3216
[Epoch 4/40] Loss: 0.3024
[Epoch 5/40] Loss: 0.2955
[Epoch 6/40] Loss: 0.2920
[Epoch 7/40] Loss: 0.2920
[Epoch 8/40] Loss: 0.2890
[Epoch 9/40] Loss: 0.2745
[Epoch 10/40] Loss: 0.2744
[Epoch 11/40] Loss: 0.2748
[Epoch 12/40] Loss: 0.2640
[Epoch 13/40] Loss: 0.2579
[Epoch 14/40] Loss: 0.2607
[Epoch 15/40] Loss: 0.2513
[Epoch 16/40] Loss: 0.2522
[Epoch 17/40] Loss: 0.2475
[Epoch 18/40] Loss: 0.2447
[Epoch 19/40] Loss: 0.2367
[Epoch 20/40] Loss: 0.2338
[Epoch 21/40] Loss: 0.2365
[Epoch 22/40] Loss: 0.2398
[Epoch 23/40] Loss: 0.2276
[Epoch 24/40] Loss: 0.2220
[Epoch 25/40] Loss: 0.2183
[Epoch 26/40] Loss: 0.2190
[Epoch 27/40] Loss: 0.2141
[Epoch 28/40] Loss: 0.2108
[Epoch 29/40] Loss: 0.2101
[Epoch 30/40] Loss: 0.2071
[Epoch 31/40] Loss: 0.2150
[Epoch 32/40] Loss: 0.2059
[Epoch 33/40] Loss: 0.2083
[Epoch 34/40] Loss: 0.1977
[Epoch 35/40] Loss: 0.2031
[Epoch 36/40] Loss: 0.1994
[Epoch 37/40] Loss: 0.1983
[Epoch 38/

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valida


------ Test 성능 (GRU + Top 7 Features) ------
MAE   : 67069.132
RMSE  : 105981.222
SMAPE : 65.000%
         영업일자           예측매출
0  2025-08-10       0.000000
1  2025-08-11   66215.394717
2  2025-08-12   73709.315003
3  2025-08-13   82086.927599
4  2025-08-14   91460.469554
..        ...            ...
78 2025-10-27  335191.784316
79 2025-10-28  335221.678180
80 2025-10-29  335251.992238
81 2025-10-30  335281.796059
82 2025-10-31  335310.639435

[83 rows x 2 columns]


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valida

**0.4대 피처까지 넣어서 실험**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random

# =========================================================
# 🔒 0) Seed 고정
# =========================================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)


# =========================================================
# 1) 평가 지표
# =========================================================
def smape(a, f):
    a = np.array(a)
    f = np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    mask = denom != 0
    return np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100

def evaluate(actual, pred):
    actual = np.array(actual)
    pred = np.array(pred)
    mae = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred)**2))
    smape_v = smape(actual, pred)
    return mae, rmse, smape_v


# =========================================================
# 2) 매출 정제
# =========================================================
def clean_sales(df):
    df['일매출'] = (
        df['일매출'].astype(str)
        .str.replace(',', '')
        .str.replace(' ', '')
        .str.strip()
    )
    df['일매출'] = pd.to_numeric(df['일매출'], errors='coerce').fillna(0)
    return df


# =========================================================
# 3) 날짜 기반 Feature
# =========================================================
def add_date_features(df):
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['weekday'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)

    df['open_hours'] = 11
    df.loc[df['weekday'] == 5, 'open_hours'] = 6
    df.loc[df['weekday'] == 6, 'open_hours'] = 0
    return df


# =========================================================
# 4) Lag / Rolling Feature (전체 생성)
# =========================================================
def add_lag_features(df):
    df['lag1'] = df['일매출'].shift(1)
    df['lag7'] = df['일매출'].shift(7)
    df['lag14'] = df['일매출'].shift(14)
    df['lag28'] = df['일매출'].shift(28)

    df['roll_mean7'] = df['일매출'].rolling(7).mean()
    df['roll_mean14'] = df['일매출'].rolling(14).mean()
    df['roll_mean28'] = df['일매출'].rolling(28).mean()

    df['roll_std7'] = df['일매출'].rolling(7).std()
    return df


# =========================================================
# 5) Dataset
# =========================================================
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# =========================================================
# 6) GRU Model
# =========================================================
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out


# =========================================================
# 7) Autoregressive Forecast
# =========================================================
def autoregressive_forecast(model, df_train, df_test, feature_cols, scaler_X, scaler_y):
    history = df_train.copy()
    predictions = []

    for i in range(len(df_test)):
        row = df_test.iloc[i].copy()
        last = history['일매출'].values

        # 필요한 Feature만 업데이트
        row['lag1'] = last[-1]
        row['lag7'] = last[-7]
        row['lag14'] = last[-14]
        row['lag28'] = last[-28]

        row['roll_mean7'] = pd.Series(last[-7:]).mean()
        row['roll_mean14'] = pd.Series(last[-14:]).mean()
        row['roll_mean28'] = pd.Series(last[-28:]).mean()

        row['roll_std7'] = pd.Series(last[-7:]).std()

        # 스케일 변환
        X_input = scaler_X.transform([row[feature_cols].values])
        X_tensor = torch.tensor(X_input, dtype=torch.float32).unsqueeze(1)

        pred_scaled = model(X_tensor).item()
        pred = scaler_y.inverse_transform([[pred_scaled]])[0][0]

        pred = max(pred, 0)
        predictions.append(pred)

        # history 업데이트
        new_row = row.copy()
        new_row['일매출'] = pred
        history = pd.concat([history, pd.DataFrame([new_row])], ignore_index=True)

    return predictions


# =========================================================
# 🔥 메인 실행
# =========================================================

# ① POS 데이터
train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test['영업일자'] = pd.to_datetime(test['영업일자'])

train = clean_sales(train)
test = clean_sales(test)

# ② 학사일정 데이터
academic = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/학사일정_정리(2325).csv')

academic['영업일자'] = pd.to_datetime(academic['date'])
academic = academic.drop(columns=['date'])

weekday_map = {'mon':0,'tue':1,'wed':2,'thu':3,'fri':4,'sat':5,'sun':6}
academic['acad_weekday'] = academic['weekday'].map(weekday_map)
academic = academic.drop(columns=['weekday'])

academic = academic.rename(columns={
    'weekend': 'acad_weekend',
    'holiday': 'acad_holiday',
    'semester': 'acad_semester',
    'seasonal': 'acad_seasonal',
    'exam': 'acad_exam',
    'ceremony': 'acad_ceremony',
    'dormitory': 'acad_dormitory'
})

train = train.merge(academic, on='영업일자', how='left')
test = test.merge(academic, on='영업일자', how='left')

# 날짜 Feature
train = add_date_features(train)
test = add_date_features(test)

# Lag / Rolling 적용
train = add_lag_features(train)
test = add_lag_features(test)

train = train.dropna().reset_index(drop=True)

# =========================================================
# 🧩 최종 Feature 12개
# =========================================================
feature_cols = [
    'lag7',
    'roll_mean7',
    'open_hours',
    'is_weekend',
    'acad_semester',
    'roll_mean14',
    'lag14',

    'lag1',
    'weekday',
    'roll_std7',
    'lag28',
    'roll_mean28'
]

# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

train[feature_cols] = scaler_X.fit_transform(train[feature_cols])
train[['일매출']] = scaler_y.fit_transform(train[['일매출']])

# GRU 학습 준비
X_train = train[feature_cols].values.reshape(-1, 1, len(feature_cols))
y_train = train['일매출'].values

dataset = SequenceDataset(X_train, y_train)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

model = GRUModel(input_dim=len(feature_cols))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# GRU 학습
for epoch in range(40):
    total_loss = 0
    for Xb, yb in loader:
        optimizer.zero_grad()
        pred = model(Xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[Epoch {epoch+1}/40] Loss: {total_loss:.4f}")

# Test autoregressive 예측
test_pred = autoregressive_forecast(model, train, test, feature_cols, scaler_X, scaler_y)

# 평가
actual_test = test['일매출'].values
mae, rmse, smape_val = evaluate(actual_test, test_pred)

print("\n------ Test 성능 (GRU + Selected 12 Features) ------")
print(f"MAE   : {mae:.3f}")
print(f"RMSE  : {rmse:.3f}")
print(f"SMAPE : {smape_val:.3f}%")

test['예측매출'] = test_pred
print(test[['영업일자', '예측매출']])


[Epoch 1/40] Loss: 0.6876
[Epoch 2/40] Loss: 0.3738
[Epoch 3/40] Loss: 0.3093
[Epoch 4/40] Loss: 0.2876
[Epoch 5/40] Loss: 0.2905
[Epoch 6/40] Loss: 0.2692
[Epoch 7/40] Loss: 0.2599
[Epoch 8/40] Loss: 0.2581
[Epoch 9/40] Loss: 0.2661
[Epoch 10/40] Loss: 0.2459
[Epoch 11/40] Loss: 0.2437
[Epoch 12/40] Loss: 0.2427
[Epoch 13/40] Loss: 0.2667
[Epoch 14/40] Loss: 0.2340
[Epoch 15/40] Loss: 0.2339
[Epoch 16/40] Loss: 0.2306
[Epoch 17/40] Loss: 0.2192
[Epoch 18/40] Loss: 0.2315
[Epoch 19/40] Loss: 0.2180
[Epoch 20/40] Loss: 0.2184
[Epoch 21/40] Loss: 0.2153
[Epoch 22/40] Loss: 0.2144
[Epoch 23/40] Loss: 0.2186
[Epoch 24/40] Loss: 0.2095
[Epoch 25/40] Loss: 0.2142
[Epoch 26/40] Loss: 0.2077
[Epoch 27/40] Loss: 0.2209
[Epoch 28/40] Loss: 0.2039
[Epoch 29/40] Loss: 0.2050
[Epoch 30/40] Loss: 0.1986
[Epoch 31/40] Loss: 0.2236
[Epoch 32/40] Loss: 0.1978
[Epoch 33/40] Loss: 0.1967
[Epoch 34/40] Loss: 0.1927
[Epoch 35/40] Loss: 0.1936
[Epoch 36/40] Loss: 0.1922
[Epoch 37/40] Loss: 0.1951
[Epoch 38/

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valida


------ Test 성능 (GRU + Selected 12 Features) ------
MAE   : 63932.352
RMSE  : 102173.916
SMAPE : 68.284%
         영업일자           예측매출
0  2025-08-10       0.000000
1  2025-08-11   58130.102684
2  2025-08-12   66530.690935
3  2025-08-13   69857.253619
4  2025-08-14   72794.453238
..        ...            ...
78 2025-10-27  349775.307387
79 2025-10-28  367116.329241
80 2025-10-29  349229.894453
81 2025-10-30  327860.464436
82 2025-10-31  306291.531900

[83 rows x 2 columns]


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valida

**0.3대 피처까지 추가**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random

# =========================================================
# 🔒 0) Seed 고정
# =========================================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)


# =========================================================
# 1) 평가 지표
# =========================================================
def smape(a, f):
    a = np.array(a)
    f = np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    mask = denom != 0
    return np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100

def evaluate(actual, pred):
    actual = np.array(actual)
    pred = np.array(pred)
    mae = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred)**2))
    smape_v = smape(actual, pred)
    return mae, rmse, smape_v


# =========================================================
# 2) 매출 정제
# =========================================================
def clean_sales(df):
    df['일매출'] = (
        df['일매출'].astype(str)
        .str.replace(',', '')
        .str.replace(' ', '')
        .str.strip()
    )
    df['일매출'] = pd.to_numeric(df['일매출'], errors='coerce').fillna(0)
    return df


# =========================================================
# 3) 날짜 Feature
# =========================================================
def add_date_features(df):
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['weekday'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)

    df['open_hours'] = 11
    df.loc[df['weekday'] == 5, 'open_hours'] = 6
    df.loc[df['weekday'] == 6, 'open_hours'] = 0
    return df


# =========================================================
# 4) Lag / Rolling Feature
# =========================================================
def add_lag_features(df):
    df['lag1'] = df['일매출'].shift(1)
    df['lag7'] = df['일매출'].shift(7)
    df['lag14'] = df['일매출'].shift(14)
    df['lag28'] = df['일매출'].shift(28)

    df['roll_mean7'] = df['일매출'].rolling(7).mean()
    df['roll_mean14'] = df['일매출'].rolling(14).mean()
    df['roll_mean28'] = df['일매출'].rolling(28).mean()

    df['roll_std7'] = df['일매출'].rolling(7).std()
    df['roll_std28'] = df['일매출'].rolling(28).std()
    return df


# =========================================================
# 5) Dataset
# =========================================================
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# =========================================================
# 6) GRU 모델
# =========================================================
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out


# =========================================================
# 7) Autoregressive 예측
# =========================================================
def autoregressive_forecast(model, df_train, df_test, feature_cols, scaler_X, scaler_y):
    history = df_train.copy()
    predictions = []

    for i in range(len(df_test)):
        row = df_test.iloc[i].copy()
        last = history['일매출'].values

        # 필수 Lag 업데이트
        row['lag1'] = last[-1]
        row['lag7'] = last[-7]
        row['lag14'] = last[-14]
        row['lag28'] = last[-28]

        # Rolling 업데이트 (필요한 것만)
        row['roll_mean7'] = pd.Series(last[-7:]).mean()
        row['roll_mean14'] = pd.Series(last[-14:]).mean()
        row['roll_mean28'] = pd.Series(last[-28:]).mean()
        row['roll_std7'] = pd.Series(last[-7:]).std()
        row['roll_std28'] = pd.Series(last[-28:]).std()

        # 스케일 변환
        X_input = scaler_X.transform([row[feature_cols].values])
        X_tensor = torch.tensor(X_input, dtype=torch.float32).unsqueeze(1)

        pred_scaled = model(X_tensor).item()
        pred = scaler_y.inverse_transform([[pred_scaled]])[0][0]
        pred = max(pred, 0)
        predictions.append(pred)

        # history 확장
        new_row = row.copy()
        new_row['일매출'] = pred
        history = pd.concat([history, pd.DataFrame([new_row])], ignore_index=True)

    return predictions


# =========================================================
# 🔥 메인 실행
# =========================================================

# ① POS 데이터
train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test['영업일자'] = pd.to_datetime(test['영업일자'])

train = clean_sales(train)
test = clean_sales(test)

# ② 학사일정 데이터
academic = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/학사일정_정리(2325).csv')

academic['영업일자'] = pd.to_datetime(academic['date'])
academic = academic.drop(columns=['date'])

weekday_map = {'mon':0,'tue':1,'wed':2,'thu':3,'fri':4,'sat':5,'sun':6}
academic['acad_weekday'] = academic['weekday'].map(weekday_map)
academic = academic.drop(columns=['weekday'])

academic = academic.rename(columns={
    'weekend': 'acad_weekend',
    'holiday': 'acad_holiday',
    'semester': 'acad_semester',
    'seasonal': 'acad_seasonal',
    'exam': 'acad_exam',
    'ceremony': 'acad_ceremony',
    'dormitory': 'acad_dormitory'
})

train = train.merge(academic, on='영업일자', how='left')
test = test.merge(academic, on='영업일자', how='left')

# 날짜 Feature
train = add_date_features(train)
test = add_date_features(test)

# Lag / Rolling 생성
train = add_lag_features(train)
test = add_lag_features(test)

train = train.dropna().reset_index(drop=True)

# =========================================================
# 📌 최종 Feature 리스트 (13개)
# =========================================================
feature_cols = [
    'lag7',
    'roll_mean7',
    'open_hours',
    'is_weekend',
    'acad_semester',
    'roll_mean14',
    'lag14',
    'lag1',
    'weekday',
    'roll_std7',
    'lag28',
    'roll_mean28',
    'roll_std28'
]

# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

train[feature_cols] = scaler_X.fit_transform(train[feature_cols])
train[['일매출']] = scaler_y.fit_transform(train[['일매출']])

# GRU 학습 준비
X_train = train[feature_cols].values.reshape(-1, 1, len(feature_cols))
y_train = train['일매출'].values

dataset = SequenceDataset(X_train, y_train)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

model = GRUModel(input_dim=len(feature_cols))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# GRU 학습
for epoch in range(40):
    total_loss = 0
    for Xb, yb in loader:
        optimizer.zero_grad()
        pred = model(Xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[Epoch {epoch+1}/40] Loss: {total_loss:.4f}")

# Test autoregressive 예측
test_pred = autoregressive_forecast(model, train, test, feature_cols, scaler_X, scaler_y)

# 평가
actual_test = test['일매출'].values
mae, rmse, smape_val = evaluate(actual_test, test_pred)

print("\n------ Test 성능 (GRU + Selected 13 Features) ------")
print(f"MAE   : {mae:.3f}")
print(f"RMSE  : {rmse:.3f}")
print(f"SMAPE : {smape_val:.3f}%")

test['예측매출'] = test_pred
print(test[['영업일자', '예측매출']])


[Epoch 1/40] Loss: 1.6275
[Epoch 2/40] Loss: 0.4884
[Epoch 3/40] Loss: 0.3618
[Epoch 4/40] Loss: 0.3168
[Epoch 5/40] Loss: 0.3117
[Epoch 6/40] Loss: 0.2968
[Epoch 7/40] Loss: 0.2906
[Epoch 8/40] Loss: 0.2806
[Epoch 9/40] Loss: 0.2758
[Epoch 10/40] Loss: 0.2755
[Epoch 11/40] Loss: 0.2568
[Epoch 12/40] Loss: 0.2515
[Epoch 13/40] Loss: 0.2516
[Epoch 14/40] Loss: 0.2506
[Epoch 15/40] Loss: 0.2512
[Epoch 16/40] Loss: 0.2438
[Epoch 17/40] Loss: 0.2648
[Epoch 18/40] Loss: 0.2359
[Epoch 19/40] Loss: 0.2334
[Epoch 20/40] Loss: 0.2430
[Epoch 21/40] Loss: 0.2330
[Epoch 22/40] Loss: 0.2209
[Epoch 23/40] Loss: 0.2259
[Epoch 24/40] Loss: 0.2178
[Epoch 25/40] Loss: 0.2189
[Epoch 26/40] Loss: 0.2146
[Epoch 27/40] Loss: 0.2352
[Epoch 28/40] Loss: 0.2167
[Epoch 29/40] Loss: 0.2101
[Epoch 30/40] Loss: 0.2141
[Epoch 31/40] Loss: 0.2144
[Epoch 32/40] Loss: 0.2129
[Epoch 33/40] Loss: 0.2056
[Epoch 34/40] Loss: 0.2084
[Epoch 35/40] Loss: 0.2179
[Epoch 36/40] Loss: 0.2044
[Epoch 37/40] Loss: 0.1993
[Epoch 38/

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valida


------ Test 성능 (GRU + Selected 13 Features) ------
MAE   : 157053.958
RMSE  : 194909.660
SMAPE : 100.800%
         영업일자           예측매출
0  2025-08-10       0.000000
1  2025-08-11   80709.071121
2  2025-08-12   94178.162377
3  2025-08-13   99308.861726
4  2025-08-14  103329.691375
..        ...            ...
78 2025-10-27  515568.353480
79 2025-10-28  549458.509952
80 2025-10-29  532052.297866
81 2025-10-30  507784.003508
82 2025-10-31  482488.542804

[83 rows x 2 columns]


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valida

**Permutation Feature Importance**

---


특정 feature 값을 무작위로 섞은 후 모델의 성능이 얼마나 저하되는 지를 측정하여 각 피처의 중요도를 평가하는 방법

**Sequential length 1에서 7로 변경**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random

# =========================================================
# 🔒 0) Seed 고정
# =========================================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# =========================================================
# 1) 평가 지표
# =========================================================
def smape(a, f):
    a = np.array(a)
    f = np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    mask = denom != 0
    return np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100

def evaluate(actual, pred):
    actual = np.array(actual)
    pred = np.array(pred)
    mae = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred)**2))
    smape_v = smape(actual, pred)
    return mae, rmse, smape_v

# =========================================================
# 2) 매출 정제
# =========================================================
def clean_sales(df):
    df['일매출'] = (
        df['일매출'].astype(str)
        .str.replace(',', '')
        .str.replace(' ', '')
        .str.strip()
    )
    df['일매출'] = pd.to_numeric(df['일매출'], errors='coerce').fillna(0)
    return df

# =========================================================
# 3) 날짜 Feature
# =========================================================
def add_date_features(df):
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['weekday'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)

    df['open_hours'] = 11
    df.loc[df['weekday'] == 5, 'open_hours'] = 6
    df.loc[df['weekday'] == 6, 'open_hours'] = 0
    return df

# =========================================================
# 4) Lag / Rolling Feature
# =========================================================
def add_lag_features(df):
    df['lag1'] = df['일매출'].shift(1)
    df['lag7'] = df['일매출'].shift(7)
    df['lag14'] = df['일매출'].shift(14)
    df['lag28'] = df['일매출'].shift(28)

    df['roll_mean7'] = df['일매출'].rolling(7).mean()
    df['roll_mean14'] = df['일매출'].rolling(14).mean()
    df['roll_mean28'] = df['일매출'].rolling(28).mean()

    df['roll_std7'] = df['일매출'].rolling(7).std()
    df['roll_std28'] = df['일매출'].rolling(28).std()
    return df

# =========================================================
# 5) Sliding Window Sequence 생성
# =========================================================
def create_sequences(df, feature_cols, seq_len=7):
    X, y = [], []
    values = df[feature_cols].values
    targets = df['일매출'].values

    for i in range(seq_len, len(df)):
        X.append(values[i-seq_len:i])
        y.append(targets[i])

    return np.array(X), np.array(y)

# =========================================================
# 6) Dataset 클래스
# =========================================================
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# =========================================================
# 7) GRU 모델
# =========================================================
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

# =========================================================
# 8) Autoregressive 예측 (seq_len=7 버전)
# =========================================================
def autoregressive_forecast(model, df_train, df_test, feature_cols, scaler_X, scaler_y, seq_len=7):
    history = df_train.copy()
    predictions = []

    for i in range(len(df_test)):
        last_block = history.iloc[-seq_len:].copy()

        # lag/rolling 업데이트
        last_vals = history['일매출'].values

        df_test.loc[df_test.index[i], 'lag1'] = last_vals[-1]
        df_test.loc[df_test.index[i], 'lag7'] = last_vals[-7]
        df_test.loc[df_test.index[i], 'lag14'] = last_vals[-14]
        df_test.loc[df_test.index[i], 'lag28'] = last_vals[-28]

        df_test.loc[df_test.index[i], 'roll_mean7'] = pd.Series(last_vals[-7:]).mean()
        df_test.loc[df_test.index[i], 'roll_mean14'] = pd.Series(last_vals[-14:]).mean()
        df_test.loc[df_test.index[i], 'roll_mean28'] = pd.Series(last_vals[-28:]).mean()
        df_test.loc[df_test.index[i], 'roll_std7'] = pd.Series(last_vals[-7:]).std()
        df_test.loc[df_test.index[i], 'roll_std28'] = pd.Series(last_vals[-28:]).std()

        # sequence 생성
        seq_data = history[feature_cols].tail(seq_len).values
        seq_scaled = scaler_X.transform(seq_data)

        X_tensor = torch.tensor(seq_scaled, dtype=torch.float32).unsqueeze(0)

        pred_scaled = model(X_tensor).item()
        pred = scaler_y.inverse_transform([[pred_scaled]])[0][0]
        pred = max(pred, 0)

        predictions.append(pred)

        new_row = df_test.iloc[i].copy()
        new_row['일매출'] = pred
        history = pd.concat([history, pd.DataFrame([new_row])], ignore_index=True)

    return predictions

# =========================================================
# 🔥 메인 실행
# =========================================================

# ① POS 데이터
train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test['영업일자'] = pd.to_datetime(test['영업일자'])

train = clean_sales(train)
test = clean_sales(test)

# ② 학사일정
academic = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/학사일정_정리(2325).csv')

academic['영업일자'] = pd.to_datetime(academic['date'])
academic = academic.drop(columns=['date'])

weekday_map = {'mon':0,'tue':1,'wed':2,'thu':3,'fri':4,'sat':5,'sun':6}
academic['acad_weekday'] = academic['weekday'].map(weekday_map)
academic = academic.drop(columns=['weekday'])

academic = academic.rename(columns={
    'weekend': 'acad_weekend',
    'holiday': 'acad_holiday',
    'semester': 'acad_semester',
    'seasonal': 'acad_seasonal',
    'exam': 'acad_exam',
    'ceremony': 'acad_ceremony',
    'dormitory': 'acad_dormitory'
})

train = train.merge(academic, on='영업일자', how='left')
test = test.merge(academic, on='영업일자', how='left')

# 날짜 Feature
train = add_date_features(train)
test = add_date_features(test)

# Lag / Rolling
train = add_lag_features(train)
test = add_lag_features(test)

train = train.dropna().reset_index(drop=True)

# =========================================================
# 전체 Feature 사용
# =========================================================
feature_cols = [
    'month','day','weekday','is_weekend','open_hours',
    'lag1','lag7','lag14','lag28',
    'roll_mean7','roll_mean14','roll_mean28',
    'roll_std7','roll_std28',
    'acad_weekday','acad_weekend','acad_holiday',
    'acad_semester','acad_seasonal','acad_exam',
    'acad_ceremony','acad_dormitory'
]

# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

train[feature_cols] = scaler_X.fit_transform(train[feature_cols])
train[['일매출']] = scaler_y.fit_transform(train[['일매출']])

# Sliding window 적용
seq_len = 7
X_train, y_train = create_sequences(train, feature_cols, seq_len)

dataset = SequenceDataset(X_train, y_train)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# GRU 모델
model = GRUModel(input_dim=len(feature_cols))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 학습
for epoch in range(40):
    total_loss = 0
    for Xb, yb in loader:
        optimizer.zero_grad()
        pred = model(Xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[Epoch {epoch+1}/40] Loss: {total_loss:.4f}")

# Test 예측
test_pred = autoregressive_forecast(
    model, train, test, feature_cols, scaler_X, scaler_y, seq_len=seq_len
)

# 평가
actual_test = test['일매출'].values
mae, rmse, smape_val = evaluate(actual_test, test_pred)

print("\n------ Test 성능 (GRU + seq_len=7 + all features) ------")
print(f"MAE   : {mae:.3f}")
print(f"RMSE  : {rmse:.3f}")
print(f"SMAPE : {smape_val:.3f}%")

test['예측매출'] = test_pred
print(test[['영업일자', '예측매출']])


[Epoch 1/40] Loss: 0.7044
[Epoch 2/40] Loss: 0.5402
[Epoch 3/40] Loss: 0.4593
[Epoch 4/40] Loss: 0.3807
[Epoch 5/40] Loss: 0.3591
[Epoch 6/40] Loss: 0.3786
[Epoch 7/40] Loss: 0.3901
[Epoch 8/40] Loss: 0.3103
[Epoch 9/40] Loss: 0.2863
[Epoch 10/40] Loss: 0.2861
[Epoch 11/40] Loss: 0.2632
[Epoch 12/40] Loss: 0.2426
[Epoch 13/40] Loss: 0.2412
[Epoch 14/40] Loss: 0.2742
[Epoch 15/40] Loss: 0.2495
[Epoch 16/40] Loss: 0.2570
[Epoch 17/40] Loss: 0.2487
[Epoch 18/40] Loss: 0.2221
[Epoch 19/40] Loss: 0.2952
[Epoch 20/40] Loss: 0.3070
[Epoch 21/40] Loss: 0.2521
[Epoch 22/40] Loss: 0.2194
[Epoch 23/40] Loss: 0.2170
[Epoch 24/40] Loss: 0.2165
[Epoch 25/40] Loss: 0.2142
[Epoch 26/40] Loss: 0.2560
[Epoch 27/40] Loss: 0.2349
[Epoch 28/40] Loss: 0.2217
[Epoch 29/40] Loss: 0.2107
[Epoch 30/40] Loss: 0.2063
[Epoch 31/40] Loss: 0.2057
[Epoch 32/40] Loss: 0.1981
[Epoch 33/40] Loss: 0.1989
[Epoch 34/40] Loss: 0.2026
[Epoch 35/40] Loss: 0.2227
[Epoch 36/40] Loss: 0.2970
[Epoch 37/40] Loss: 0.2952
[Epoch 38/

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valida


------ Test 성능 (GRU + seq_len=7 + all features) ------
MAE   : 86997.578
RMSE  : 119664.619
SMAPE : 82.136%
         영업일자           예측매출
0  2025-08-10   82527.216782
1  2025-08-11   97816.066946
2  2025-08-12   78776.152235
3  2025-08-13   81920.898103
4  2025-08-14   76378.985555
..        ...            ...
78 2025-10-27  363517.960507
79 2025-10-28  362186.543065
80 2025-10-29  337051.686841
81 2025-10-30  316724.550185
82 2025-10-31  232842.129931

[83 rows x 2 columns]


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [ ]:
# ============================================================
# 🔍 Permutation Feature Importance (MAE, RMSE, SMAPE 모두 계산)
# ============================================================

import copy

def permutation_importance_full(model, train_df, test_df, feature_cols, scaler_X, scaler_y):
    """
    학습된 GRU 모델 기반으로 전체 피처 중요도를
    MAE / RMSE / SMAPE 세 지표 기준으로 계산한다.
    """

    # ---- baseline prediction ----
    baseline_pred = autoregressive_forecast(
        model, train_df, test_df, feature_cols, scaler_X, scaler_y
    )
    actual = test_df['일매출'].values
    base_mae, base_rmse, base_smape = evaluate(actual, baseline_pred)

    print("🔥 Baseline 성능:")
    print(f"MAE   : {base_mae:.4f}")
    print(f"RMSE  : {base_rmse:.4f}")
    print(f"SMAPE : {base_smape:.4f}%")
    print("\n=== 🔍 Permutation Feature Importance ===")

    results = []

    for col in feature_cols:
        print(f"\n▶ 피처 permute: {col}")

        # test_df 복사 후 피처 셔플
        temp_test = test_df.copy()
        shuffled = temp_test[col].sample(frac=1, random_state=42).values
        temp_test[col] = shuffled

        # 예측 수행
        pred = autoregressive_forecast(
            model, train_df, temp_test, feature_cols, scaler_X, scaler_y
        )

        mae, rmse, smape = evaluate(actual, pred)

        results.append([
            col,
            mae - base_mae,
            rmse - base_rmse,
            smape - base_smape,
        ])

    # 표 형태로 출력
    result_df = pd.DataFrame(results, columns=[
        'feature', 'ΔMAE', 'ΔRMSE', 'ΔSMAPE'
    ])

    # 영향 큰 순 정렬
    result_df = result_df.sort_values(by='ΔMAE', ascending=False)

    print("\n===== 📊 Permutation Feature Importance (정렬) =====")
    print(result_df.to_string(index=False))

    return result_df


# 실행
pfi_result = permutation_importance_full(
    model, train, test, feature_cols, scaler_X, scaler_y
)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valida

🔥 Baseline 성능:
MAE   : 99026.5134
RMSE  : 133099.4975
SMAPE : 79.8085%

=== 🔍 Permutation Feature Importance ===

▶ 피처 permute: month


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valida


▶ 피처 permute: day


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valida

...

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valida


▶ 피처 permute: acad_dormitory


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valida


===== 📊 Permutation Feature Importance (정렬) =====
       feature         ΔMAE        ΔRMSE    ΔSMAPE
    open_hours 32602.503071 31668.273796 12.072287
  acad_holiday 14640.786176 31905.301544  6.795305
 acad_semester  7302.371984  2310.836918  9.141599
       weekday  4254.657685  4113.548247  6.643542
  acad_weekend  3484.401871  -748.486356  9.171888
  acad_weekday  3001.711851  2670.451296  6.516941
 acad_ceremony  2731.011680  4131.805125  1.489644
    is_weekend  1009.406591 -1041.185904  7.534515
    roll_std28     0.000000     0.000000  0.000000
     roll_std7     0.000000     0.000000  0.000000
          lag3     0.000000     0.000000  0.000000
          lag2     0.000000     0.000000  0.000000
          lag1     0.000000     0.000000  0.000000
          lag7     0.000000     0.000000  0.000000
         lag28     0.000000     0.000000  0.000000
         lag14     0.000000     0.000000  0.000000
    roll_mean7     0.000000     0.000000  0.000000
 acad_seasonal     0.000000    

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valida

# **Prophet**

In [ ]:
!pip uninstall -y prophet pystan
!pip install prophet==1.1.5
!pip install cmdstanpy==1.2.2

In [ ]:
!pip install numpy==1.26.4
!pip install cmdstanpy==1.2.2
!pip install prophet==1.1.5

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet

# =========================================
# 1) 매출 text → 숫자 변환
# =========================================
def clean_sales(df):
    df['일매출'] = (
        df['일매출'].astype(str)
        .str.replace(',', '')
        .str.replace(' ', '')
        .str.strip()
    )
    df['일매출'] = pd.to_numeric(df['일매출'], errors='coerce').fillna(0)
    return df

# =========================================
# 2) 날짜 기반 feature
# =========================================
def add_date_features(df):
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['weekday'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)

    df['open_hours'] = 11
    df.loc[df['weekday'] == 5, 'open_hours'] = 6
    df.loc[df['weekday'] == 6, 'open_hours'] = 0

    return df

# =========================================
# 3) Prophet regressor 목록
# =========================================
feature_cols = [
    'month','day','weekday','is_weekend','open_hours'
]

# =========================================
# 4) Prophet 모델 생성
# =========================================
def create_prophet_model():
    model = Prophet(
        weekly_seasonality=True,
        yearly_seasonality=False,
        daily_seasonality=False
    )

    # 날짜 기반 feature 추가
    for col in feature_cols:
        model.add_regressor(col)

    return model

# =========================================
# 5) Prophet 학습
# =========================================
def train_prophet_with_features(train_df):
    df = train_df.rename(columns={'영업일자': 'ds', '일매출': 'y'})
    model = create_prophet_model()
    model.fit(df)
    return model

# =========================================
# 6) Prophet 예측
# =========================================
def predict_with_features(model, test_df):
    future = test_df[['영업일자']].rename(columns={'영업일자': 'ds'})

    # future에 날짜 기반 feature 붙이기
    future = future.merge(
        test_df[['영업일자'] + feature_cols].rename(columns={'영업일자': 'ds'}),
        on='ds', how='left'
    )

    forecast = model.predict(future)
    out = forecast[['ds', 'yhat']].rename(columns={'ds': '영업일자', 'yhat': '예측매출'})
    out['예측매출'] = out['예측매출'].clip(lower=0)

    return out

# =========================================
# 7) 평가 지표
# =========================================
def smape(a, f):
    a = np.array(a)
    f = np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    mask = denom != 0
    return np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100

def evaluate(test_df, pred_df):
    merged = test_df.merge(pred_df, on='영업일자', how='left')
    actual = merged['일매출'].astype(float)
    pred = merged['예측매출'].astype(float)

    mae = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred)**2))
    smape_v = smape(actual, pred)

    return mae, rmse, smape_v

# =========================================================
# 🚀 메인 실행
# =========================================================
train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test  = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test['영업일자']  = pd.to_datetime(test['영업일자'])

# 매출 정제
train = clean_sales(train)
test = clean_sales(test)

# 날짜 feature 생성
train = add_date_features(train)
test = add_date_features(test)

# Prophet용 train df 준비
train_prophet_df = train.rename(columns={'영업일자': 'ds'})

# Prophet 학습
model = train_prophet_with_features(train_prophet_df)

# test 예측
pred_df = predict_with_features(model, test)

# 평가
mae, rmse, smape_v = evaluate(test, pred_df)

print("------ Test 성능 (Prophet + Date Features) ------")
print(f"MAE   : {mae:.3f}")
print(f"RMSE  : {rmse:.3f}")
print(f"SMAPE : {smape_v:.3f}%")

print("\n------ Test 예측 결과 ------")
print(pred_df)


DEBUG:cmdstanpy:input tempfile: /tmp/tmpitarh4yo/b1d__tgx.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpitarh4yo/d8x9lii0.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=12079', 'data', 'file=/tmp/tmpitarh4yo/b1d__tgx.json', 'init=/tmp/tmpitarh4yo/d8x9lii0.json', 'output', 'file=/tmp/tmpitarh4yo/prophet_modeldl1fprmb/prophet_model-20251117041102.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
04:11:02 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
04:11:02 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


------ Test 성능 (Prophet + Date Features) ------
MAE   : 86106.591
RMSE  : 127189.367
SMAPE : 69.043%

------ Test 예측 결과 ------
         영업일자           예측매출
0  2025-08-10       0.000000
1  2025-08-11  262224.541372
2  2025-08-12  284151.887590
3  2025-08-13  279155.280465
4  2025-08-14  272175.937166
..        ...            ...
78 2025-10-27  270321.975423
79 2025-10-28  292249.321641
80 2025-10-29  287252.714516
81 2025-10-30  280273.371217
82 2025-10-31  235706.156548

[83 rows x 2 columns]


**Prophet-학사일정추가**

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet

# =========================================
# 1) 매출 text → 숫자 변환
# =========================================
def clean_sales(df):
    df['일매출'] = (
        df['일매출'].astype(str)
        .str.replace(',', '')
        .str.replace(' ', '')
        .str.strip()
    )
    df['일매출'] = pd.to_numeric(df['일매출'], errors='coerce').fillna(0)
    return df

# =========================================
# 2) 날짜 기반 feature
# =========================================
def add_date_features(df):
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['weekday'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)

    df['open_hours'] = 11   # 월~금 11시간
    df.loc[df['weekday'] == 5, 'open_hours'] = 6  # 토 6시간
    df.loc[df['weekday'] == 6, 'open_hours'] = 0  # 일 0시간

    return df

# =========================================
# 3) Prophet regressor 목록 정의
# =========================================
date_feature_cols = ['month', 'day', 'weekday', 'is_weekend', 'open_hours']

academic_feature_cols = [
    'acad_weekday', 'acad_weekend', 'acad_holiday',
    'acad_semester', 'acad_seasonal', 'acad_exam',
    'acad_ceremony', 'acad_dormitory'
]

all_features = date_feature_cols + academic_feature_cols

# =========================================
# 4) Prophet 모델 생성
# =========================================
def create_prophet_model():
    model = Prophet(
        weekly_seasonality=True,
        yearly_seasonality=False,
        daily_seasonality=False
    )
    for col in all_features:
        model.add_regressor(col)
    return model

# =========================================
# 5) Prophet 학습
# =========================================
def train_prophet_with_features(train_df):
    df = train_df.rename(columns={'영업일자': 'ds', '일매출': 'y'})
    model = create_prophet_model()
    model.fit(df)
    return model

# =========================================
# 6) Prophet 예측
# =========================================
def predict_with_features(model, test_df):
    future = test_df[['영업일자']].rename(columns={'영업일자': 'ds'})

    # feature merge
    f = future.merge(
        test_df[['영업일자'] + all_features].rename(columns={'영업일자': 'ds'}),
        on='ds', how='left'
    )

    forecast = model.predict(f)
    out = forecast[['ds', 'yhat']].rename(columns={'ds': '영업일자', 'yhat': '예측매출'})
    out['예측매출'] = out['예측매출'].clip(lower=0)  # 음수 방지
    return out

# =========================================
# 7) 평가 지표
# =========================================
def smape(a, f):
    a = np.array(a)
    f = np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    mask = denom != 0
    return np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100

def evaluate(test_df, pred_df):
    merged = test_df.merge(pred_df, on='영업일자', how='left')
    actual = merged['일매출'].astype(float)
    pred = merged['예측매출'].astype(float)

    mae = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred)**2))
    smape_v = smape(actual, pred)

    return mae, rmse, smape_v

# =========================================================
# 🚀 메인 실행
# =========================================================

# --------------------------
# ① 매출 데이터 로드
# --------------------------
train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test  = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test['영업일자']  = pd.to_datetime(test['영업일자'])

train = clean_sales(train)
test = clean_sales(test)

# --------------------------
# ② 학사일정 로드 + 전처리
# --------------------------
academic = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/학사일정_정리(2325).csv')

academic['영업일자'] = pd.to_datetime(academic['date'])
academic = academic.drop(columns=['date'])

# 요일 문자열 → 숫자 매핑
weekday_map = {'mon':0,'tue':1,'wed':2,'thu':3,'fri':4,'sat':5,'sun':6}
academic['acad_weekday'] = academic['weekday'].map(weekday_map)
academic = academic.drop(columns=['weekday'])

# 컬럼명 충돌 방지용 변경
academic = academic.rename(columns={
    'weekend': 'acad_weekend',
    'holiday': 'acad_holiday',
    'semester': 'acad_semester',
    'seasonal': 'acad_seasonal',
    'exam': 'acad_exam',
    'ceremony': 'acad_ceremony',
    'dormitory': 'acad_dormitory'
})

# --------------------------
# ③ 학사일정 merge
# --------------------------
train = train.merge(academic, on='영업일자', how='left')
test = test.merge(academic, on='영업일자', how='left')

# --------------------------
# ④ 날짜 피처 추가
# --------------------------
train = add_date_features(train)
test = add_date_features(test)

# --------------------------
# ⑤ Prophet 학습
# --------------------------
train_prophet_df = train.rename(columns={'영업일자': 'ds'})
model = train_prophet_with_features(train_prophet_df)

# --------------------------
# ⑥ Prophet 예측
# --------------------------
pred_df = predict_with_features(model, test)

# --------------------------
# ⑦ 성능 평가
# --------------------------
mae, rmse, smape_v = evaluate(test, pred_df)

print("------ Test 성능 (Prophet + Date + Academic Features) ------")
print(f"MAE   : {mae:.3f}")
print(f"RMSE  : {rmse:.3f}")
print(f"SMAPE : {smape_v:.3f}%")

print("\n------ Test 예측 결과 ------")
print(pred_df)


DEBUG:cmdstanpy:input tempfile: /tmp/tmpitarh4yo/y7cp_98t.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpitarh4yo/u7qifog7.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=64782', 'data', 'file=/tmp/tmpitarh4yo/y7cp_98t.json', 'init=/tmp/tmpitarh4yo/u7qifog7.json', 'output', 'file=/tmp/tmpitarh4yo/prophet_modelvyxgocbl/prophet_model-20251117044215.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
04:42:15 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
04:42:16 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


------ Test 성능 (Prophet + Date + Academic Features) ------
MAE   : 57618.848
RMSE  : 77050.926
SMAPE : 70.834%

------ Test 예측 결과 ------
         영업일자           예측매출
0  2025-08-10       0.000000
1  2025-08-11  116215.906932
2  2025-08-12  139530.486628
3  2025-08-13  135079.447162
4  2025-08-14  126542.251634
..        ...            ...
78 2025-10-27  357629.111487
79 2025-10-28  380943.691184
80 2025-10-29  376492.651718
81 2025-10-30  367955.456189
82 2025-10-31  324032.552929

[83 rows x 2 columns]
